In [ ]:
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import numpy as np

Using Local Linearization Accuracy to find the spikiness of the data. However, due to the fact that Euclidean distances are used in such high dimensions, the metric proved to be unreliable to find out gemoetric data. By nature of the fact that this heuristic is sensitive to the exact topology of the data, we concluded that applying this method after dimensionality reduction would end in a loss of context and thus invalid measurements.

In [ ]:
"""
Our objective is to somehow concretely capture spikiness of data via some nice geometric benchmark. Measuring curvature directly is
highly nontrivial both in terms of theory and computation -- determinants of Riemannian tensors(?) -- so I wouldn't risk spending much time on it. Here's a nice heuristic to replace
it. In the sklearn library there is a method called LocallyLinearEmbedding -- look it up.
Take the weight matrices calculated by LLE, and compute the error -- high error indicates that some vector v_i doesn't lie on the "closest" subspace. This means that there is high "spikiness" at v_i wrt its knn.
We can iterate this test over subsamples of the set of vectors.
"""

# Computes the weights of LLA
def compute_weights(X,d_size, d_dim, n_neig):
  knn = NearestNeighbors(n_neighbors = n_neig)
  knn.fit(X)
  distances, indices = knn.kneighbors(X)
  W = np.zeros((d_size, d_size))
  for i in range(d_size):
      M = X[indices[i]] - X[i]
      N = np.dot(M, M.T)
      N = N + np.identity(N.shape[0]) * 1e-4
      w_i = np.linalg.solve(N, np.ones(N.shape[0]))
      s = w_i.sum()
      w_i = w_i / s
      for j in range(n_neig):
        W[i,indices[i][j]] = w_i[j]
  return W


# Computes the loss
def compute_loss(X,W, i):
  predicted = np.dot(W[i],X)
  return np.linalg.norm(predicted-X[i])

#testing functions

dataset_size = 40
vector_dim = 1500
X = np.random.rand(dataset_size,vector_dim)
X[:,-1] = 0
X[-1,:] = np.random.rand(1,vector_dim)
X[-1,-1] = 1
X = X * 15000
W_sol = compute_weights(X,dataset_size, vector_dim, 4)

for n in range(dataset_size):
  val = compute_loss(X,W_sol,n)
  print(val)
  print(" ")

5.266700782914898e-10
 
5.272597585521529e-10
 
5.381592023847141e-10
 
5.384207979744263e-10
 
5.228351265559139e-10
 
5.247723138385663e-10
 
5.703139784190434e-10
 
5.247466289702294e-10
 
5.214223837587664e-10
 
5.330433557792088e-10
 
5.229781855433636e-10
 
5.262757011099447e-10
 
5.374461502521197e-10
 
5.279522486725361e-10
 
5.358084585632211e-10
 
5.263712580595411e-10
 
5.268596959680681e-10
 
5.247431210053167e-10
 
5.285211570987014e-10
 
5.346079065932145e-10
 
5.351814229736999e-10
 
5.272140031546506e-10
 
5.239316775605073e-10
 
5.373070872900222e-10
 
5.273577276245402e-10
 
5.361498123640529e-10
 
5.229993101777679e-10
 
5.297988939296607e-10
 
5.189664119388021e-10
 
5.330862123513542e-10
 
5.381641451459462e-10
 
5.264394305676166e-10
 
5.239540719804781e-10
 
5.250244352257774e-10
 
5.23230425400984e-10
 
5.272470771334919e-10
 
5.27117893433508e-10
 
5.235347919216579e-10
 
5.291022001103917e-10
 
5.200552425894505e-10
 


We attempted to run a test on a few points lying on the subspace $R^{n-1}$, and one point with the last coordinate $1$. The algorithm was unable to distinguish between the points on the lower dimensional subspace and the spike.